In [17]:
import arcpy
import os
import pandas as pd

In [18]:
BASE_DIR = r"C:\Tutor_GIS\Python a tareas comunes en GIS"
CSV_PATH = os.path.join(BASE_DIR, "Sexo_por_Poblacion.csv")
GDB_IN = os.path.join(BASE_DIR, "Datos_Entrada.gdb")
GDB_OUT = os.path.join(BASE_DIR, "Datos_Salida.gdb")

In [20]:
arcpy.env.workspace = GDB_OUT
arcpy.env.overwriteOutput = True
print(f"📁 Workspace configurado en: {arcpy.env.workspace}")

📁 Workspace configurado en: C:\Tutor_GIS\Python a tareas comunes en GIS\Datos_Salida.gdb


In [21]:
# Nombre del Feature Class de provincias
PROVINCIAS_FC = os.path.join(GDB_IN, "Provincia")
OUT_FEATURE = os.path.join(GDB_OUT, "Provincias_Poblacion")

In [22]:
# ========= LEER CSV =========
print("📥 Leyendo CSV de población...")
df = None
encodings = ["utf-8", "latin1", "cp1252"]

for enc in encodings:
    try:
        df = pd.read_csv(CSV_PATH, encoding=enc)
        print(f"✅ CSV leído correctamente con encoding → {enc}")
        break
    except UnicodeDecodeError:
        continue

if df is None:
    raise ValueError("❌ No se pudo leer el CSV con los encodings probados.")

📥 Leyendo CSV de población...
✅ CSV leído correctamente con encoding → latin1


In [23]:
# Limpieza preventiva de texto
df["Provincia"] = df["Provincia"].astype(str).str.strip().str.upper()

In [24]:
# Guardar versión limpia del CSV
CSV_LIMPIO = os.path.join(GDB_OUT, "Sexo_por_Poblacion_LIMPIO.csv")
df.to_csv(CSV_LIMPIO, index=False, encoding="latin1")

print("\nVista previa del CSV limpio:")
print(df.head())


Vista previa del CSV limpio:
           Provincia  Hombres  Mujeres      Total
0  DISTRITO NACIONAL  490,423  538,687  1,029,110
1               AZUA  124,146  116,063    240,209
2            BAORUCO   55,629   53,088    108,717
3           BARAHONA  101,378   99,508    200,886
4            DAJABÓN   37,876   36,934     74,810


In [25]:
# ========= EXPORTAR CSV A GDB COMO TABLA =========
print("📤 Convirtiendo CSV a tabla en la geodatabase de salida...")

tabla_temp = os.path.join(GDB_OUT, "Tabla_Poblacion")
if arcpy.Exists(tabla_temp):
    arcpy.management.Delete(tabla_temp)

📤 Convirtiendo CSV a tabla en la geodatabase de salida...


In [26]:
# Convertimos el CSV limpio a tabla dentro de la GDB de salida
arcpy.conversion.TableToTable(CSV_LIMPIO, GDB_OUT, "Tabla_Poblacion")
print(f"✅ Tabla temporal creada correctamente en: {tabla_temp}")

✅ Tabla temporal creada correctamente en: C:\Tutor_GIS\Python a tareas comunes en GIS\Datos_Salida.gdb\Tabla_Poblacion


In [27]:
# ========= CREAR LAYER TEMPORAL Y HACER JOIN =========
print("🔗 Ejecutando Join entre provincias y tabla de población...")

layer_temp = "Provincia_Layer"
if arcpy.Exists(layer_temp):
    arcpy.management.Delete(layer_temp)

🔗 Ejecutando Join entre provincias y tabla de población...


In [28]:
# Crear capa temporal desde el Feature Class
arcpy.management.MakeFeatureLayer(PROVINCIAS_FC, layer_temp)

# Campos comunes
campo_fc = "NOM_PRO"
campo_tabla = "Provincia"

In [29]:
# Aplicar Join
arcpy.management.AddJoin(layer_temp, campo_fc, tabla_temp, campo_tabla, "KEEP_COMMON")

<Result 'Provincia_Layer'>

In [30]:
# Exportar el resultado del Join
if arcpy.Exists(OUT_FEATURE):
    arcpy.management.Delete(OUT_FEATURE)

arcpy.conversion.FeatureClassToFeatureClass(layer_temp, GDB_OUT, "Provincias_Poblacion")
print(f"✅ Join completado y exportado a: {OUT_FEATURE}")

✅ Join completado y exportado a: C:\Tutor_GIS\Python a tareas comunes en GIS\Datos_Salida.gdb\Provincias_Poblacion


In [31]:
# ========= LIMPIEZA =========
arcpy.management.Delete(layer_temp)
print("🧹 Capa temporal eliminada. Proceso finalizado correctamente 🎉")

print("\n--- RESUMEN FINAL ---")
print(f"🗂 Tabla creada: {tabla_temp}")
print(f"🗺  Capa final creada: {OUT_FEATURE}")
print(f"📍 Todas las salidas se guardaron en: {GDB_OUT}")

🧹 Capa temporal eliminada. Proceso finalizado correctamente 🎉

--- RESUMEN FINAL ---
🗂 Tabla creada: C:\Tutor_GIS\Python a tareas comunes en GIS\Datos_Salida.gdb\Tabla_Poblacion
🗺  Capa final creada: C:\Tutor_GIS\Python a tareas comunes en GIS\Datos_Salida.gdb\Provincias_Poblacion
📍 Todas las salidas se guardaron en: C:\Tutor_GIS\Python a tareas comunes en GIS\Datos_Salida.gdb
